In [ ]:
from google.colab import drive

import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)


In [ ]:
# drive.mount('/content/gdrive')

In [ ]:
# print(os.getcwd())
# !ls
# !cd gdrive

In [ ]:
# ! cd gdrive
# ! ls

In [ ]:
class Opt:
    def __init__(
        self,
        n_epochs: int = 200,
        batch_size: int = 64,
        lr: float = 0.0002,
        b1: float = 0.5,
        b2: float = 0.999,
        n_cpu: int = 8,
        latent_dim: int = 100,
        n_classes: int = 10,
        img_size: int = 32,
        channels: int = 1,
        sample_interval: int = 400,
    ):
        r"""A class which contains all the required inputs for the model

        n_epochs: number of epochs of training

        batch_size: size of batches

        lr: adam learning rate

        b1: adam decay of first order momentum of gradient

        b2: adam decay of first order momentum of gradient

        n_cpu: number of cpu threads during batch generation

        latent_dim: dimensionality of latent space

        n_classes: number of classes for dataset

        img_size: size of each image channels

        channels: number of image channels

        sample_interval: interval between image sampling
        """
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.lr = lr
        self.b1 = b1
        self.b2 = b2
        self.n_cpu = n_cpu
        self.latent_dim = latent_dim
        self.n_classes = n_classes
        self.img_size = img_size
        self.channels = channels
        self.sample_interval = sample_interval


opt = Opt()

img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

In [ ]:
# Loss functions
adversarial_loss = torch.nn.MSELoss()

In [ ]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [ ]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
os.makedirs("../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)


In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)



In [ ]:
# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

Streaming output truncated to the last 5000 lines.
[Epoch 194/200] [Batch 629/938] [D loss: 0.091021] [G loss: 0.665015]
[Epoch 194/200] [Batch 630/938] [D loss: 0.089791] [G loss: 0.760533]
[Epoch 194/200] [Batch 631/938] [D loss: 0.075576] [G loss: 0.646845]
[Epoch 194/200] [Batch 632/938] [D loss: 0.066520] [G loss: 0.799733]
[Epoch 194/200] [Batch 633/938] [D loss: 0.062046] [G loss: 0.789569]
[Epoch 194/200] [Batch 634/938] [D loss: 0.063950] [G loss: 0.703621]
[Epoch 194/200] [Batch 635/938] [D loss: 0.061681] [G loss: 0.907683]
[Epoch 194/200] [Batch 636/938] [D loss: 0.067001] [G loss: 0.655434]
[Epoch 194/200] [Batch 637/938] [D loss: 0.100119] [G loss: 0.992377]
[Epoch 194/200] [Batch 638/938] [D loss: 0.168643] [G loss: 0.338767]
[Epoch 194/200] [Batch 639/938] [D loss: 0.070079] [G loss: 0.863181]
[Epoch 194/200] [Batch 640/938] [D loss: 0.106523] [G loss: 0.945453]
[Epoch 194/200] [Batch 641/938] [D loss: 0.155369] [G loss: 0.346448]
[Epoch 194/200] [Batch 642/938] [D loss

In [ ]:
from google.colab import files

In [ ]:
files.download('images')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/images_final.zip /content/images

  adding: content/images/ (stored 0%)
  adding: content/images/27600.png (deflated 4%)
  adding: content/images/105600.png (deflated 4%)
  adding: content/images/185200.png (deflated 4%)
  adding: content/images/159600.png (deflated 4%)
  adding: content/images/103200.png (deflated 4%)
  adding: content/images/178000.png (deflated 5%)
  adding: content/images/158400.png (deflated 4%)
  adding: content/images/46400.png (deflated 3%)
  adding: content/images/25600.png (deflated 4%)
  adding: content/images/112000.png (deflated 4%)
  adding: content/images/18000.png (deflated 5%)
  adding: content/images/85600.png (deflated 4%)
  adding: content/images/22800.png (deflated 4%)
  adding: content/images/31200.png (deflated 4%)
  adding: content/images/53600.png (deflated 4%)
  adding: content/images/35200.png (deflated 4%)
  adding: content/images/154000.png (deflated 4%)
  adding: content/images/161600.png (deflated 4%)
  adding: content/images/106400.png (deflated 4%)
  adding: content/ima